**Секція 1. Логістична регресія з нуля.**

Будемо крок за кроком будувати модель лог регресії з нуля для передбачення, чи буде врожай більше за 80 яблук (задача подібна до лекційної, але на класифікацію).

Давайте нагадаємо основні формули для логістичної регресії.

### Функція гіпотези - обчислення передбачення у логістичній регресії:

$$
\hat{y} = \sigma(x W^T + b) = \frac{1}{1 + e^{-(x W^T + b)}}
$$

Де:
- $ \hat{y} $ — це ймовірність "позитивного" класу.
- $ x $ — це вектор (або матриця для набору прикладів) вхідних даних.
- $ W $ — це вектор (або матриця) вагових коефіцієнтів моделі.
- $ b $ — це зміщення (bias).
- $ \sigma(z) $ — це сигмоїдна функція активації.

### Як обчислюється сигмоїдна функція:

Сигмоїдна функція $ \sigma(z) $ має вигляд:

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

Ця функція перетворює будь-яке дійсне значення $ z $ в інтервал від 0 до 1, що дозволяє інтерпретувати вихід як ймовірність для логістичної регресії.

### Формула функції втрат для логістичної регресії (бінарна крос-ентропія):

Функція втрат крос-ентропії оцінює, наскільки добре модель передбачає класи, порівнюючи передбачені ймовірності $ \hat{y} $ із справжніми мітками $ y $. Формула наступна:

$$
L(y, \hat{y}) = - \left[ y \cdot \log(\hat{y}) + (1 - y) \cdot \log(1 - \hat{y}) \right]
$$

Де:
- $ y $ — це справжнє значення (мітка класу, 0 або 1).
- $ \hat{y} $ — це передбачене значення (ймовірність).



1.
Тут вже наведений код для ініціювання набору даних в форматі numpy. Перетворіть `inputs`, `targets` на `torch` тензори. Виведіть результат на екран.

In [49]:
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

In [16]:
# Вхідні дані (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')

# Таргети (apples > 80)
targets = np.array([[0],
                    [1],
                    [1],
                    [0],
                    [1]], dtype='float32')

In [17]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[0.],
        [1.],
        [1.],
        [0.],
        [1.]])


2. Ініціюйте ваги `w`, `b` для моделі логістичної регресії потрібної форми зважаючи на розмірності даних випадковими значеннями з нормального розподілу. Лишаю тут код для фіксації `random_seed`.

In [18]:
torch.random.manual_seed(1)

In [19]:
# Weights and biases
w = torch.randn(3, requires_grad=True)
b = torch.randn(1, requires_grad=True)
print(w)
print(b)

tensor([0.6614, 0.2669, 0.0617], requires_grad=True)
tensor([0.6213], requires_grad=True)


3. Напишіть функцію `model`, яка буде обчислювати функцію гіпотези в логістичній регресії і дозволяти робити передбачення на основі введеного рядка даних і коефіцієнтів в змінних `w`, `b`.

  **Важливий момент**, що функція `model` робить обчислення на `torch.tensors`, тож для математичних обчислень використовуємо фукнціонал `torch`, наприклад:
  - обчсилення $e^x$: `torch.exp(x)`
  - обчсилення $log(x)$: `torch.log(x)`
  - обчислення середнього значення вектору `x`: `torch.mean(x)`

  Використайте функцію `model` для обчислення передбачень з поточними значеннями `w`, `b`.Виведіть результат обчислень на екран.

  Проаналізуйте передбачення. Чи не викликають вони у вас підозр? І якщо викликають, то чим це може бути зумовлено?

In [20]:
# Визначаємо модель
def model(x, w, b, threshold=0.5):
    z = torch.matmul(x, w) + b
    probs = 1 / (1 + torch.exp(-z))
    preds = (probs > threshold).float() 
    return preds, probs

In [21]:
# Генеруємо передбачення
preds, probs = model(inputs, w, b)
print(preds)

tensor([1., 1., 1., 1., 1.])


In [22]:
print(probs)

tensor([1., 1., 1., 1., 1.], grad_fn=<MulBackward0>)


Моделька явно предиктись все як один класс і при чому ймоврініть чомусь завжди рівна 100%. Можливо підхід з рандомними вагами і зсувом треба переглянути.

4. Напишіть функцію `binary_cross_entropy`, яка приймає на вхід передбачення моделі `predicted_probs` та справжні мітки в даних `true_labels` і обчислює значення втрат (loss)  за формулою бінарної крос-ентропії для кожного екземпляра та вертає середні втрати по всьому набору даних.
  Використайте функцію `binary_cross_entropy` для обчислення втрат для поточних передбачень моделі.

In [23]:
def binary_cross_entropy(predicted_probs, true_labels):
    # Уникаємо обчислень log(0) або log(1) шляхом використання невеликого значення eps
    eps = 1e-7
    predicted_probs = torch.clamp(predicted_probs, eps, 1 - eps)
    
    loss = - (true_labels * torch.log(predicted_probs) + (1 - true_labels) * torch.log(1 - predicted_probs))
    
    return loss.mean()

In [27]:
loss = binary_cross_entropy(probs, targets)
print(loss.item())

6.376953601837158


5. Зробіть зворотнє поширення помилки і виведіть градієнти за параметрами `w`, `b`. Проаналізуйте їх значення. Як гадаєте, чому вони саме такі?

In [28]:
# Обчислимо gradients
loss.backward()

# Градієнти вагів
print(w)
print(w.grad)

tensor([0.6614, 0.2669, 0.0617], requires_grad=True)
tensor([0., 0., 0.])


In [29]:
# Gradients for bias
print(b)
print(b.grad)

tensor([0.6213], requires_grad=True)
tensor([0.])


**Що сталось?**

В цій задачі, коли ми ініціювали значення випадковими значеннями з нормального розподілу, насправді ці значення не були дуже гарними стартовими значеннями і привели до того, що градієнти стали дуже малими або навіть рівними нулю (це призводить до того, що градієнти "зникають"), і відповідно при оновленні ваг у нас не буде нічого змінюватись. Це називається `gradient vanishing`. Це відбувається через **насичення сигмоїдної функції активації.**

У нашій задачі ми використовуємо сигмоїдну функцію активації, яка має такий вигляд:

   $$
   \sigma(z) = \frac{1}{1 + e^{-z}}
   $$


Коли значення $z$ дуже велике або дуже мале, сигмоїдна функція починає "насичуватись". Це означає, що для великих позитивних $z$ сигмоїда наближається до 1, а для великих негативних — до 0. В цих діапазонах градієнти починають стрімко зменшуватись і наближаються до нуля (бо градієнт - це похідна, похідна на проміжку функції, де вона паралельна осі ОХ, дорівнює 0), що робить оновлення ваг неможливим.

![](https://editor.analyticsvidhya.com/uploads/27889vaegp.png)

У логістичній регресії $ z = x \cdot w + b $. Якщо ваги $w, b$ - великі, значення $z$ також буде великим, і сигмоїда перейде в насичену область, де градієнти дуже малі.

Саме це сталося в нашій задачі, де великі випадкові значення ваг викликали насичення сигмоїдної функції. Це в свою чергу призводить до того, що під час зворотного поширення помилки (backpropagation) модель оновлює ваги дуже повільно або зовсім не оновлює. Це називається проблемою **зникнення градієнтів** (gradient vanishing problem).

**Що ж робити?**
Ініціювати ваги маленькими значеннями навколо нуля. Наприклад ми можемо просто в існуючій ініціалізації ваги розділити на 1000. Можна також використати інший спосіб ініціалізації вагів - інформація про це [тут](https://www.geeksforgeeks.org/initialize-weights-in-pytorch/).

Як це робити - показую нижче. **Виконайте код та знову обчисліть передбачення, лосс і виведіть градієнти.**

А я пишу пояснення, чому просто не зробити

```
w = torch.randn(1, 3, requires_grad=True)/1000
b = torch.randn(1, requires_grad=True)/1000
```

Нам потрібно, аби тензори вагів були листовими (leaf tensors).

1. **Що таке листовий тензор**
Листовий тензор — це тензор, який був створений користувачем безпосередньо і з якого починається обчислювальний граф. Якщо такий тензор має `requires_grad=True`, PyTorch буде відслідковувати всі операції, виконані над ним, щоб правильно обчислювати градієнти під час навчання.

2. **Чому ми використовуємо `w.data` замість звичайних операцій**
Якщо ми просто виконали б операції, такі як `(w - 0.5) / 100`, ми б отримали **новий тензор**, який вже не був би листовим тензором, оскільки ці операції створюють **новий** тензор, а не модифікують існуючий.

  Проте, щоб залишити наші тензори ваги `w` та зміщення `b` листовими і продовжити можливість відстеження градієнтів під час тренування, ми використовуємо атрибут `.data`. Цей атрибут дозволяє **виконувати операції in-place (прямо на існуючому тензорі)** без зміни самого об'єкта тензора. Отже, тензор залишається листовим, і PyTorch може коректно обчислювати його градієнти.

3. **Чому важливо залишити тензор листовим**
Якщо тензор більше не є листовим (наприклад, через проведення операцій, що створюють нові тензори), ви не зможете отримати градієнти за допомогою `w.grad` чи `b.grad` після виклику `loss.backward()`. Це може призвести до втрати можливості оновлення параметрів під час тренування моделі. В нашому випадку ми хочемо, щоб тензори `w` та `b` накопичували градієнти, тому вони повинні залишатись листовими.

**Висновок:**
Ми використовуємо `.data`, щоб виконати операції зміни значень на ваги і зміщення **in-place**, залишаючи їх листовими тензорами, які можуть накопичувати градієнти під час навчання. Це дозволяє коректно працювати механізму зворотного поширення помилки (backpropagation) і оновлювати ваги моделі.

5. Виконайте код та знову обчисліть передбачення, лосс і знайдіть градієнти та виведіть всі ці тензори на екран.

In [30]:
torch.random.manual_seed(1)
w = torch.randn(3, requires_grad=True)  # Листовий тензор
b = torch.randn(1, requires_grad=True)     # Листовий тензор

# in-place операції
w.data = w.data / 1000
b.data = b.data / 1000

In [31]:
w.data

tensor([6.6135e-04, 2.6692e-04, 6.1677e-05])

In [32]:
# Генеруємо передбачення
preds, probs = model(inputs, w, b)
print(preds)

tensor([1., 1., 1., 1., 1.])


In [33]:
# Рахуємо функцію втрат
loss = binary_cross_entropy(probs, targets)
print(loss.item())

0.6857479810714722


In [34]:
# Обчислимо gradients
loss.backward(retain_graph=True)

# Градієнти вагів
print(w)
print(w.grad)

tensor([6.6135e-04, 2.6692e-04, 6.1677e-05], requires_grad=True)
tensor([-6.6817, -6.7453, -4.3082])


In [35]:
# Gradients for bias
print(b)
print(b.grad)

tensor([0.0006], requires_grad=True)
tensor([-0.0794])


6. Напишіть алгоритм градієнтного спуску, який буде навчати модель з використанням написаних раніше функцій і виконуючи оновлення ваг. Алгоритм має включати наступні кроки:

  1. Генерація прогнозів
  2. Обчислення втрат
  3. Обчислення градієнтів (gradients) loss-фукнції відносно ваг і зсувів
  4. Налаштування ваг шляхом віднімання невеликої величини, пропорційної градієнту (`learning_rate` домножений на градієнт)
  5. Скидання градієнтів на нуль

Виконайте градієнтний спуск протягом 1000 епох, обчисліть фінальні передбачення і проаналізуйте, чи вони точні?

In [36]:
learning_rate = 1e-5

for epoch in range(1000):
    # Генеруємо передбачення
    preds, probs = model(inputs, w, b)
    
    # Рахуємо функцію втрат
    loss = binary_cross_entropy(probs, targets)
    
    # Обчислюємо градієнти
    loss.backward()
    
    # Оновлюємо ваги та зміщення
    with torch.no_grad():
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad
        
        # Скидаємо градієнти
        w.grad.zero_()
        b.grad.zero_()
    
    # Виводимо втрати для кожної 10 епохи
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

Epoch 0, Loss: 0.6857479810714722
Epoch 10, Loss: 0.6780029535293579
Epoch 20, Loss: 0.6753175854682922
Epoch 30, Loss: 0.6742188334465027
Epoch 40, Loss: 0.6737637519836426
Epoch 50, Loss: 0.6735718250274658
Epoch 60, Loss: 0.673488438129425
Epoch 70, Loss: 0.6734501123428345
Epoch 80, Loss: 0.6734302639961243
Epoch 90, Loss: 0.673418402671814
Epoch 100, Loss: 0.6734099388122559
Epoch 110, Loss: 0.6734029650688171
Epoch 120, Loss: 0.6733965873718262
Epoch 130, Loss: 0.6733908653259277
Epoch 140, Loss: 0.6733851432800293
Epoch 150, Loss: 0.6733798384666443
Epoch 160, Loss: 0.6733747124671936
Epoch 170, Loss: 0.6733695864677429
Epoch 180, Loss: 0.6733646988868713
Epoch 190, Loss: 0.6733599305152893
Epoch 200, Loss: 0.6733553409576416
Epoch 210, Loss: 0.6733508110046387
Epoch 220, Loss: 0.6733464598655701
Epoch 230, Loss: 0.6733422875404358
Epoch 240, Loss: 0.6733381152153015
Epoch 250, Loss: 0.6733341217041016
Epoch 260, Loss: 0.6733302474021912
Epoch 270, Loss: 0.6733264923095703
Epoch

In [37]:
# Виведемо фінальні значення вагів та зміщення
print("Final weights:", w)
print("Final bias:", b)

Final weights: tensor([0.0032, 0.0008, 0.0013], requires_grad=True)
Final bias: tensor([0.0007], requires_grad=True)


In [39]:
final_preds, final_probs = model(inputs, w, b)
print("Final predictions:", final_preds)

Final predictions: tensor([1., 1., 1., 1., 1.])


Видно, що 1000 епох для навчання забагато, так як після 100 епох, функція втрат майже не міняється.  А такоє ми все ще погано предіктимо, тому всі дані асайнимо до 1 класу.

**Секція 2. Створення лог регресії з використанням функціоналу `torch.nn`.**

Давайте повторно реалізуємо ту ж модель, використовуючи деякі вбудовані функції та класи з PyTorch.

Даних у нас буде побільше - тож, визначаємо нові масиви.

In [92]:
# Вхідні дані (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')

# Таргети (apples > 80)
targets = np.array([[0],
                    [1],
                    [1],
                    [0],
                    [1],
                    [0],
                    [1],
                    [1],
                    [0],
                    [1],
                    [0],
                    [1],
                    [1],
                    [0],
                    [1]], dtype='float32')

7. Завантажте вхідні дані та мітки в PyTorch тензори та з них створіть датасет, який поєднує вхідні дані з мітками, використовуючи клас `TensorDataset`. Виведіть перші 3 елементи в датасеті.



In [93]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [94]:
# Визначаємо dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[0.],
         [1.],
         [1.]]))

8. Визначте data loader з класом **DataLoader** для підготовленого датасету `train_ds`, встановіть розмір батчу на 5 та увімкніть перемішування даних для ефективного навчання моделі. Виведіть перший елемент в дата лоадері.

In [95]:
# Визначаємо data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
next(iter(train_dl))

[tensor([[102.,  43.,  37.],
         [ 87., 134.,  58.],
         [ 91.,  88.,  64.],
         [ 73.,  67.,  43.],
         [ 73.,  67.,  43.]]),
 tensor([[0.],
         [1.],
         [1.],
         [0.],
         [0.]])]

9. Створіть клас `LogReg` для логістичної регресії, наслідуючи модуль `torch.nn.Module` за прикладом в лекції (в частині про FeedForward мережі).

  У нас модель складається з лінійної комбінації вхідних значень і застосування фукнції сигмоїда. Тож, нейромережа буде складатись з лінійного шару `nn.Linear` і використання активації `nn.Sigmid`. У створеному класі мають бути реалізовані методи `__init__` з ініціалізацією шарів і метод `forward` для виконання прямого проходу моделі через лінійний шар і функцію активації.

  Створіть екземпляр класу `LogReg` в змінній `model`.

In [96]:
class LogReg(nn.Module):
    # Initialize the layers
    def __init__(self):
        super(LogReg, self).__init__()
        self.linear = nn.Linear(3, 1)
        self.sigmoid = nn.Sigmoid() # Activation function

    # Perform the computation
    def forward(self, x):
        z = self.linear(x)
        probs = self.sigmoid(z)
        return probs

In [97]:
model = LogReg()

10. Задайте оптимізатор `Stockastic Gradient Descent` в змінній `opt` для навчання моделі логістичної регресії. А також визначіть в змінній `loss` функцію втрат `binary_cross_entropy` з модуля `torch.nn.functional` для обчислення втрат моделі. Обчисліть втрати для поточних передбачень і міток, а потім виведіть їх. Зробіть висновок, чи моделі вдалось навчитись?

In [98]:
opt = torch.optim.SGD(model.parameters(), 1e-5)
loss_fn = F.binary_cross_entropy

In [99]:
loss = loss_fn(model(inputs), targets)
print(loss.item())

9.865747451782227


In [100]:
model(inputs)

tensor([[8.3396e-07],
        [1.6896e-08],
        [4.0041e-08],
        [7.8229e-09],
        [5.5783e-07],
        [8.3396e-07],
        [1.6896e-08],
        [4.0041e-08],
        [7.8229e-09],
        [5.5783e-07],
        [8.3396e-07],
        [1.6896e-08],
        [4.0041e-08],
        [7.8229e-09],
        [5.5783e-07]], grad_fn=<SigmoidBackward0>)

In [101]:
targets

tensor([[0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.]])

Виглядає так, як наче ми усі предікти робимо як клас 0. А отде моделька навчилась погано.

11. Візьміть з лекції функцію для тренування моделі з відстеженням значень втрат і навчіть щойно визначену модель на 1000 епохах. Виведіть після цього графік зміни loss, фінальні передбачення і значення таргетів.

In [102]:
# Модифікована функцію fit для відстеження втрат
def fit_return_loss(num_epochs, model, loss_fn, opt, train_dl):
    losses = []
    for epoch in range(num_epochs):
        # Ініціалізуємо акумулятор для втрат
        total_loss = 0

        for xb, yb in train_dl:
            # Генеруємо передбачення
            pred = model(xb)

            # Обчислюємо втрати
            loss = loss_fn(pred, yb)

            # Очищення градієнтів
            opt.zero_grad()
            loss.backward()  # Обчислення градієнтів
            opt.step()  # Оновлення ваг

            # Накопичуємо втрати
            total_loss += loss.item()

        # Обчислюємо середні втрати для епохи
        avg_loss = total_loss / len(train_dl)
        losses.append(avg_loss)

        # Виводимо підсумок епохи
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss:.4f}')
    return losses

In [103]:
# Train the model for 1000 epochs
loss = fit_return_loss(1000, model, loss_fn, opt, train_dl)

Epoch [10/1000], Loss: 9.7613
Epoch [20/1000], Loss: 6.1834
Epoch [30/1000], Loss: 2.7785
Epoch [40/1000], Loss: 0.7222
Epoch [50/1000], Loss: 0.4441
Epoch [60/1000], Loss: 0.3591
Epoch [70/1000], Loss: 0.2637
Epoch [80/1000], Loss: 0.1229
Epoch [90/1000], Loss: 0.3501
Epoch [100/1000], Loss: 0.3670
Epoch [110/1000], Loss: 0.1062
Epoch [120/1000], Loss: 0.1469
Epoch [130/1000], Loss: 0.2364
Epoch [140/1000], Loss: 0.3252
Epoch [150/1000], Loss: 0.2954
Epoch [160/1000], Loss: 0.2315
Epoch [170/1000], Loss: 0.2357
Epoch [180/1000], Loss: 0.2913
Epoch [190/1000], Loss: 0.2352
Epoch [200/1000], Loss: 0.3707
Epoch [210/1000], Loss: 0.2307
Epoch [220/1000], Loss: 0.2387
Epoch [230/1000], Loss: 0.1530
Epoch [240/1000], Loss: 0.2302
Epoch [250/1000], Loss: 0.0971
Epoch [260/1000], Loss: 0.0971
Epoch [270/1000], Loss: 0.1486
Epoch [280/1000], Loss: 0.2296
Epoch [290/1000], Loss: 0.3243
Epoch [300/1000], Loss: 0.1519
Epoch [310/1000], Loss: 0.1821
Epoch [320/1000], Loss: 0.2326
Epoch [330/1000],

In [104]:
with torch.no_grad():
    final_preds = model(inputs)
    print("Final Predictions:", final_preds)

Final Predictions: tensor([[0.4796],
        [0.6554],
        [0.9944],
        [0.0032],
        [0.9824],
        [0.4796],
        [0.6554],
        [0.9944],
        [0.0032],
        [0.9824],
        [0.4796],
        [0.6554],
        [0.9944],
        [0.0032],
        [0.9824]])


In [105]:
targets

tensor([[0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.]])

Тут вже ситуація краща, виглядає так що моделька потрохи навчилась предіктити два класи.